# RQAE and IQAE

Present notebook reviews the **Iterative Quantum Amplitude Estimation** (**MLAE**) and **Real Quantum Amplitude Estimation** (RQAE) algorithms which were implemented into the modules *iterative_quantum_ae* and *real_quantum_ae* respectively within the package *AE* of library *QQuantLib* (**QQuantLib/AE/iterative_quantum_ae.py** and **real_quantum_ae**). 

Present notebook and modules are based on the following references:

* *Grinko, D., Gacon, J., Zoufal, C. & Woerner, S.*. Iterative Quantum Amplitude Estimation. npj Quantum Information 7, 2021. https://www.nature.com/articles/s41534-021-00379-1

* *Manzano, A., Musso, D., Leitao, A., Gómez, A., Vázquez, C., Nogueiras, M. & Ordóñez, G.*. Real Quantum Amplitude Estimation. Preprint.

In [1]:
import sys
sys.path.append("../../")

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import qat.lang.AQASM as qlm

In [3]:
%matplotlib inline

In [4]:
#This cell loads the QLM solver.
#QLMaaS == False -> uses PyLinalg
#QLMaaS == True -> try to use LinAlg (for using QPU as CESGA QLM one)
from QQuantLib.utils.qlm_solver import get_qpu
QLMaaS = False
linalg_qpu = get_qpu(QLMaaS)

Using PyLinalg


In [5]:
#See 01_DataLoading_Module_Use for the use of this function
from QQuantLib.utils.data_extracting import get_results
from QQuantLib.utils.utils import bitfield_to_int

## 1. Oracle generation

Before doing any amplitude estimation we want to load some data into the quantum circuit, as this step is only auxiliary to see how the algorithm works, we are just going to load a discrete probability distribution. In this case we will have a circuit with $n=3$ qubits which makes a total of $N = 2^n = 8$ states. The discrete probability distribution that we are going to load is:
$$p_d = \dfrac{(0,1,2,3,4,5,6,7)}{0+1+2+3+4+5+6+7+8}.$$


In [6]:
n = 3
N = 2**n
x = np.arange(N)
probability = x/np.sum(x)

Note that this probability distribution is properly normalised. For loading this probability into the quantum circuit we will use the function *load_probability* from **QQuantLib/DL/data_loading** module. The state that we are going to get is:
    $$|\Psi\rangle = \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{1}|1\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].$$

In [7]:
from QQuantLib.DL.data_loading import load_probability

In [8]:
oracle = load_probability(probability)

For more information about loading data into the quantum circuit see the notebook *01_DataLoading_Module_Use*.

## 2. IQAE class.

The problem of amplitude estimation is the following. Given an oracle:

$$\mathcal{0}|0\rangle = |\Psi\rangle = \sqrt{a}|\Psi_0\rangle +\sqrt{1-a}|\Psi_1\rangle,$$

where $|\Psi_0\rangle$ and $|\Psi_1\rangle$ are orthogonal states, we want to estimate $a$.

We have implemented and python class called **IQAE** into the **QQuantLib/AE/iterative_quantum_ae** module that allows us to use the **IQAE** algorithm.

### Definition of the problem

In [9]:
#import the class
from QQuantLib.AE.iterative_quantum_ae import IQAE

For showing how our class and the algorithm works, we will define the following amplitude estimation problem:
$$
    \begin{array}{l}
    &\mathcal{O}\longrightarrow \mathcal{P}.\\
    & |\Psi\rangle \longrightarrow \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{1}|1\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].\\
    & \sqrt{a}|\Psi_0\rangle \longrightarrow \dfrac{\sqrt{1}}{\sqrt{0+1+2+3+4+5+6+7+8}}|1\rangle.\\
    & \sqrt{1-a}|\Psi_1\rangle \longrightarrow \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].\\
    \end{array}
$$
The target state, in this case is $|1\rangle$. It's binary representation is $001$. This has to be passed to the target variable as a list. Moreover we have to provide the list of qubits where we are acting, in this case is just $[0,1,2]$, the whole register.

In [10]:
target = [0,0,1]
index = [0,1,2]
a = probability[bitfield_to_int(target)]

iqae = IQAE(oracle,target = target,index = [0,1,2])

### Execution

The only relevant methods for the end user are *run* and *display_information*. First, the method *run* returns an upper and lower bound estimates of the probability $a$: $(a_{\min},a_{\max})$. The output is in the form of a list. The function has three inputs:
* epsilon: the precision. Ensures that the width of the interval is, at most, $2\epsilon$.
* N: the number of shots on each iteration of the algorithm.
* alpha: the accuracy. Ensures that the probability of $a$ not laying within the given interval is, at most, $\alpha$.

In [11]:
epsilon = 0.001
N = 100
alpha = 0.05
bounds = iqae.run(epsilon = epsilon,N = N, alpha = alpha)

Now we will check weather the real value of $a$ lays within the bounds. Most of the times (at least with 95% in this case) it will be the case.

In [12]:
print("Bounds: ",bounds)
print("Real value: ",a)

Bounds:  [0.03546125490577309, 0.036182217734121]
Real value:  0.03571428571428571


In [13]:
if (a>=bounds[0])&(a<=bounds[1]):
    print("Correct")
else:
    print("Incorrect")

Correct


Second, the method *display_information* displays some informamtion of the inner workings of the method. The function has the same three inputs as in the previous case.

In [14]:
iqae.display_information(epsilon = epsilon,N = N, alpha = alpha)

-------------------------------------------------------------
Maximum number of rounds:  9
Maximum number of shots per round needed:  618
Maximum number of amplifications:  333
Maximum number of calls to the oracle:  297622
-------------------------------------------------------------


## 3. RQAE class

The new problem of amplitude estimation is the following. Given an oracle:

$$\mathcal{0}|0\rangle = |\Psi\rangle = a|\Psi_0\rangle +\sqrt{1-a^2}|\Psi_1\rangle,$$

where $|\Psi_0\rangle$ and $|\Psi_1\rangle$ are orthogonal states, we want to estimate the real parameter $a$. Note that there is a slight difference with the previous case, now $a$ is not within a square root. Moreover, it is a real parameter, this means that it can take values in the domain $[-1,1]$.

We have implemented and python class called **RQAE** into the **QQuantLib/AE/real_quantum_ae** module that allows us to use the **RQAE** algorithm.

### Definition of the problem

In [15]:
#import the class
from QQuantLib.AE.real_quantum_ae import RQAE

For showing how our class and the algorithm works, we will define the following amplitude estimation problem:
$$
    \begin{array}{l}
    &\mathcal{O}\longrightarrow \mathcal{P}.\\
    & |\Psi\rangle \longrightarrow \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{1}|1\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].\\
    & a|\Psi_0\rangle \longrightarrow \dfrac{\sqrt{1}}{\sqrt{0+1+2+3+4+5+6+7+8}}|1\rangle.\\
    & \sqrt{1-a^2}|\Psi_1\rangle \longrightarrow \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].\\
    \end{array}
$$
The target state, in this case is $|1\rangle$. It's binary representation is $001$. This has to be passed to the target variable as a list. Moreover we have to provide the list of qubits where we are acting, in this case is just $[0,1,2]$, the whole register.

In [16]:
target = [0,0,1]
index = [0,1,2]
a = np.sqrt(probability[bitfield_to_int(target)])

rqae = RQAE(oracle,target = [0,0,1],index = [0,1,2])

Note again that a is now defined as an amplitude, that is why we take it's square root.

### Execution

The only relevant methods for the end user are *run* and *display_information*. First, the method *run* returns an upper and lower bound estimates of the amplitude $a$: $(a_{\min},a_{\max})$. The output is in the form of a list. The function has three inputs:
* q: the ratio of amplifications between iterations. It's value has to be bigger than one. Rule of thumbs $q=2$.
* epsilon: the precision. Ensures that the width of the interval is, at most, $2\epsilon$.
* alpha: the accuracy. Ensures that the probability of $a$ not laying within the given interval is, at most, $\alpha$.

In [17]:
q = 2
epsilon = 0.001
gamma = 0.05
bounds = rqae.run(q = q,epsilon = epsilon, gamma = gamma)

Now we will check weather the real value of $a$ lays within the bounds. Most of the times (at least with 95% in this case) it will be the case.

In [18]:
print("Bounds: ",bounds)
print("Real value: ",a)

Bounds:  [0.18811034064807367, 0.18997788536667365]
Real value:  0.1889822365046136


In [19]:
if (a>=bounds[0])&(a<=bounds[1]):
    print("Correct")
else:
    print("Incorrect")

Correct


Second, the method *display_information* displays some informamtion of the inner workings of the method. The function has the same three inputs as in the previous case.

In [20]:
rqae.display_information(q = q,epsilon = epsilon, gamma = gamma)

-------------------------------------------------------------
Maximum number of amplifications:  98
Maximum number of rounds:  9
Number of shots per round:  556
Maximum number of calls to the oracle:  164298
-------------------------------------------------------------
